In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans

In [2]:
movies_df = pd.read_csv(r'C:\Users\ethic\Desktop\movie.csv')
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Removing the parentheses
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
#Removing the years from the 'title' column
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
#Applying the strip function to get rid of any ending whitespace characters that may have appeared
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [4]:
movies_df['genres'] = movies_df.genres.str.split('|')
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [5]:
moviesWithGenres_df= movies_df.copy()
#For every row in the dataframe, iterate through the list of genres and place a 1 into the corresponding column
for index, row in movies_df.iterrows():
    for genre in row['genres']:
        moviesWithGenres_df.at[index, genre] = 1
#Filling in the NaN values with 0 to show that a movie doesn't have that column's genre
moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
moviesWithGenres_df= moviesWithGenres_df.drop("genres",axis=1)

In [7]:
x = moviesWithGenres_df.iloc[:,3:]
y = moviesWithGenres_df.iloc[:,1] 

In [8]:
km=KMeans(n_clusters=15,init='k-means++', n_init=10, max_iter=500 )
km.fit(x)
pred = km.predict(x)

In [9]:
movies_df["pred"] = pred

In [10]:
movies_df_copy = movies_df.copy()

In [11]:
#Convert - list to string
def listToString(s):  
    str1 = ""    
    for ele in s:  
        str1 = str1 + ele + " "     
    return str1 

In [12]:
for i in range(0,len(movies_df)):
    movies_df_copy.genres[movies_df_copy.index == i] = listToString(list(movies_df.genres[movies_df.index == i])[0])
               

C:\Users\ethic\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [15]:
#Recommendation Function

def recommendation(movie):
    trial = movies_df_copy.pred[movies_df_copy.title ==movie].values[0]
    clus = movies_df_copy[movies_df_copy.pred == int(trial)]
    clus["ix"] = range(0,len(clus))
    cv = CountVectorizer()
    count_matrix = cv.fit_transform(clus.genres)
    cosine_sim = cosine_similarity(count_matrix)
    for i in range(0,len(clus)):
        cosine_sim[i][i] = 1
    def get_title_from_index(index):
        return clus[clus.index == index]["title"].values[0]
    def get_index_from_title(title):
        return clus[clus.title == title]["ix"].values[0]
    def get_year_from_index(index):
        return clus[clus.index == index]["year"].values[0]
    movie_user_likes =movie
    movie_index = get_index_from_title(movie_user_likes)
    similar_movies = list(enumerate(cosine_sim[movie_index]))
    dist=[]
    for i in range(0,len(similar_movies)):
        s = similar_movies[i][1]
        m = clus.index[i]
        dist.append((m,s))
    similar_movies = dist
    sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)[0:10]
    
    print("Movie names similar to"+"''"+movie_user_likes+"''"+" are:\n")
    for k in range(0,len(clus)):
        st=clus.iloc[k,1]
        if st.startswith(movie[0:7]):
            print(clus.iloc[k,1],clus.iloc[k,3])    
             
    print("\n")  
    i=0
    print("Top 5 Recommended movies for "+"''"+movie_user_likes+"''"+" are:\n")
    for element in sorted_similar_movies[1:10]:
        print(get_title_from_index(int(element[0])),get_year_from_index(element[0]))
        i=i+1
        if i>5:
            break

In [18]:
recommendation('Superman')

Movie names similar to''Superman'' are:

Superman 1978
Superman III 1983
Superman IV: The Quest for Peace 1987
Superman Returns 2006
Superman vs. The Elite 2012
Superman/Doomsday 2007
Superman/Batman: Public Enemies 2009
Superman/Shazam!: The Return of Black Adam (DC Showcase Original Shorts Collection) 2010
Superman Unbound 2013


Top 5 Recommended movies for ''Superman'' are:

Star Wars: Episode IV - A New Hope 1977
Stargate 1994
Demolition Man 1993
Star Wars: Episode V - The Empire Strikes Back 1980
Star Wars: Episode VI - Return of the Jedi 1983
Star Trek III: The Search for Spock 1984


C:\Users\ethic\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
#def counts(j):
 #   test_kmeans = movies_df[movies_df.pred == j]
 #   for i in range(2,len(test_kmeans.columns)):
 #       a.append(sum(test_kmeans.iloc[:,i]))
 #       b.append(test_kmeans.columns[i])
 #   c = pd.DataFrame(a,b,columns= ['count'])
 #   d = c.sort_values(by='count',ascending=False)
  #return(d.head(3))

In [ ]:
#c=[]
#for i in range(0,13):
 #   c.append(counts(i))
